# Computing initial and terminal states

In this tutorial, you will learn how to use the [GPCCAEstimator](https://cellrank.readthedocs.io/en/stable/api/cellrank.estimators.GPCCA.html) to...

- compute macrostates of cell state changes.
- visualize a coarse-grained transition matrix among these macrostates.
- automatically and manually classify macrostates as initial, terminal or intermediate.

<div class="alert alert-info">
Important

If you use the `GPCCAEstimator` in your research, plese do not forget to cite both CellRank as well as the original GPCCA papers: <cite cite-date="reuter:18,reuter:19"></cite>. See ["how to cite us"](https://cellrank.readthedocs.io/en/stable/citing.html) for more info. 

</div>

This tutorial does not depend on the kernel you used to compute a cell-cell transiton matrix. No matter whether you used the [VelocityKernel](https://cellrank.readthedocs.io/en/stable/api/cellrank.kernels.VelocityKernel.html), the [PseudotimeKernel](https://cellrank.readthedocs.io/en/stable/api/cellrank.kernels.PseudotimeKernel.html), the [CytoTRACEKernel](https://cellrank.readthedocs.io/en/stable/api/cellrank.kernels.CytoTRACEKernel.html), or any other cellrank kernel to compute transition probabilities, you may always follow the steps outlined in this tutorial to compute initial and terminal states. This is explained in more detail in our [getting started](https://cellrank.readthedocs.io/en/stable/getting_started.html) tutorial and summarized in the figure below: 

<img src="_static/img/cellrank_structure.jpg" width="800" alt="The GPCCA estimator is independent of the kernel you used to compute a cell-cell transition matrix. ">

**Fig** | CellRank is a modular framework where every kernel may be used to compute initial and terminal states. 

<div class="alert alert-info">
Important

If you want to run this on your own data, you will need...

- a scRNA-seq dataset for which you computed a cell-cell transition matrix using any CellRank kernel. For each kernel, there exist individual tutorials that teach you how to use them. 

</div>

This tutorial notebook can be downloaded using the following [link](https://github.com/theislab/cellrank_notebooks/raw/master/tutorials/initial_terminal_states.ipynb).

<div class="alert alert-warning">

Warning

This is an example for a warning. 
    
</div>

## Import packages & data

In [1]:
import sys

if "google.colab" in sys.modules:
    !pip install -q git+https://github.com/theislab/cellrank@dev

In [2]:
import scvelo as scv
import scanpy as sc
import cellrank as cr

scv.settings.verbosity = 3
scv.settings.set_figure_params("scvelo")
cr.settings.verbosity = 2

Read AnnData object from disk (or download) and reconstruct a [Precomputed Kernel](https://cellrank.readthedocs.io/en/stable/api/cellrank.kernels.PrecomputedKernel.html) from it. The transition matrix stored in this kernel was computed in the [velocity tutorial](https://cellrank.readthedocs.io/en/stable/cellrank_meets_rna_velocity.html). In this tutorial, we focus on working with the transition matrix, rather than computing it. 

In [9]:
adata = sc.read('datasets/ML_2022-06-23_pancreas_with_transition_matrix.h5ad')
pk = cr.kernels.PrecomputedKernel(adata, backward=False)
print(pk)

PrecomputedKernel[origin="adata.obsp['T_fwd']"]


<div class="alert alert-info">
Important

The `PrecomputedKernel` allows you to input any transition matrix to a CellRank `kernel` object; it is essentially an iterface between CellRank's estimators and external methods which give you transition matrices. We use it here because we have precomputed the transition matrix in another tutorial and want to use it here for further analysis. Using a `PrecomputeKernel` allows you to make use of CellRank's estimator functionalities, no matter how your transition matrix has been computed!


</div>

## Initialize an estimator 

Estimators allow you to analyze cell-state dynamics in CellRank. We initialize a `GPCCAEstimator` here by passing the kernel object. This works for *every* kernel. 

In [12]:
g = cr.estimators.GPCCA(pk)
print(g)

GPCCA[n=2531, kernel=PrecomputedKernel[origin="adata.obsp['T_fwd']"]]


## What's next? 

In this tutorial, you learned how to use CellRank to compute a transition matrix based on the CytoTRACE pseudotime and how it can be visualized in low dimensions. The real power of CellRank comes in when you use estimators to analyze the transition matrix directly, rather than projecting it. For the next steps, we recommend...

- going through the [initial & terminal states tutorial](https://cellrank.readthedocs.io/en/stable/initial_terminal.html) to learn how to use the transition matrix to automatically identify initial and terminal states. 
- taking a look at the [full API](https://cellrank.readthedocs.io/en/stable/api.html) to learn about parameter values you can use to adapt these computations to your data. 
- reading the original CytoTRACE publication and considering limitations of this appraoch <cite data-cite="gulati:20"></cite>. 

If you encounter any bugs in the code, our if you have suggestions for new features, please open an [issue](https://github.com/theislab/cellrank/issues/new/choose). If you have a general question or something you would like to discuss with us, please post on the [scverse discourse](https://discourse.scverse.org/c/ecosytem/cellrank/40). 

## Package versions

In [14]:
cr.logging.print_versions()

cellrank==1.5.1+gc04c690e.d20220610 scanpy==1.7.2 anndata==0.8.0 numpy==1.21.4 numba==0.51.2 scipy==1.5.3 pandas==1.3.3 pygpcca==1.0.2 scikit-learn==0.24.0 statsmodels==0.12.1 python-igraph==0.8.3 scvelo==0.2.4 pygam==0.8.0 matplotlib==3.3.3 seaborn==0.11.0
